# Project Euler

## Problem 51

### Prime digit replacements

In [65]:
import itertools

limit_pow = 8
limit = 10**limit_pow

def prime_sieve(n):
    sieve = [True] * (int(n/2))
    for i in range(3, int(n**0.5)+1, 2):
        if sieve[int(i/2)]:
            sieve[int(i*i/2)::i] = [False] * int((n - i*i - 1) / (2 * i) + 1)
    return [2] + [2 * i + 1 for i in range(1, int(n/2)) if sieve[i]]

primes = prime_sieve(limit)

def create_number(n, d):
    return int(n.replace('*', str(d)))
    
def create_base(n, digits):
    s = str(n)
    base = ''
    digs = set([s[d-1] for d in digits])
    if len(digs) != 1:
        return base
    l = 0
    for d in digits:
        base += s[l:d-1] + '*'
        l = d
    base += s[l:]
    return base
    
def prime_family(n):
    prime_family = []
    start = 0
    if n[0] == '*':
        start = 1
    for d in range(start,10):
        candidate = create_number(n, d)
        if candidate in primes:
            prime_family.append(candidate)
    
    return prime_family

size = 8
solved = False

combinations = {}
for i in range(1, limit_pow+1):
    if i not in combinations:
        combinations[i] = []
    for k in range(1,i):    
        combinations[i].extend([list(j) for j in list(itertools.combinations(range(1,i), k))])

while not solved:
    bases = {}
    for n in primes:
        for d in combinations[len(str(n))]:
            base = create_base(n, d)
            if base != '':
                if base in bases:
                    bases[base].add(n)
                    if len(bases[base]) >= size:
                        solved = True
                        break
                else:
                    bases[base] = set([n])
        if solved:
            break

bases[base]

{121313, 222323, 323333, 424343, 525353, 626363, 828383, 929393}

## Problem 52

### Permuted multiples

In [79]:
def digits(n):
    return sorted(list(str(n)))

def multiple_digits(n):
    l = []
    for i in range(1,7):
        l.append(digits(i*n))
    return l
        
def check(l):
    valid = True
    for i in range(0,len(l)-1):
        if l[i] != l[i+1]:
            valid = False
            break
    return valid

n = 1
while True:
    if check(multiple_digits(n)):
        break
    n += 1
    
n

142857

## Problem 53

### Combinatoric selections

In [89]:
import math

def combinations(n,r):
    return int(math.factorial(n)/(math.factorial(r)*math.factorial(n-r)))

def count_combinations_for_n(n,limit):
    a = 0
    for r in range(1,n+1):
        if combinations(n,r) >= limit:
            a += 1
    return a

a = 0
for n in range(1,101):
    a += count_combinations_for_n(n, 1000000)
    
a

4075

## Problem 54

### Poker hands

In [186]:
with open("p054_poker.txt") as f:
    s = f.read()
    
games = s.split('\n')
hands = {}

card_values = ['2', '3', '4', '5', '6', '7', '8', '9', 'T', 'J', 'Q', 'K', 'A']

for i in range(0, len(games)):
    cards = games[i].split(' ')
    hands[i] = {}
    hands[i][1] = cards[0:5]
    hands[i][2] = cards[5:10]
    
del hands[1000]

def is_royal_flush(hand):
    required_values = set(card_values[8:13])
    suites = set()
    values = set()
    for card in hand:
        values.add(card[0])
        suites.add(card[1])
    return len(suites) == 1 and required_values == values

def is_straight_flush(hand):
    suites = set()
    values = set()
    min_index = 13
    for card in hand:
        values.add(card[0])
        suites.add(card[1])
        i = card_values.index(card[0])
        if i < min_index:
            min_index = i
    if len(suites) != 1:
        return False
    return values == set(card_values[min_index:min_index+5])

def is_four_of_a_kind(hand):
    values = []
    for card in hand:
        values.append(card[0])
    for v in set(values):
        if values.count(v) == 4:
            return (True, v)
    return (False, 0)

def is_full_house(hand):
    values = []
    for card in hand:
        values.append(card[0])
    if len(set(values)) == 2: # either this, or four of a kind
        for v in set(values):
            if values.count(v) == 3:
                v1 = v
            if values.count(v) == 2:
                v2 = v
        return (True, v1, v2)
    return (False, 0, 0)

def is_flush(hand):
    suites = set()
    for card in hand:
        suites.add(card[1])
    return len(suites) == 1

def is_straight(hand):
    values = set()
    min_index = 13
    for card in hand:
        values.add(card[0])
        i = card_values.index(card[0])
        if i < min_index:
            min_index = i
    return values == set(card_values[min_index:min_index+5])

def is_three_of_a_kind(hand):
    values = []
    for card in hand:
        values.append(card[0])
    for v in set(values):
        if values.count(v) == 3:
            return (True, v)
    return (False, 0)

def is_two_pairs(hand):
    values = []
    pairs = []
    for card in hand:
        values.append(card[0])
    for v in set(values):
        if values.count(v) == 2:
            pairs.append(v)
    if len(pairs) == 2:
        return (True, sorted(pairs, reverse=True))
    return (False, [])

def is_pair(hand):
    values = []
    for card in hand:
        values.append(card[0])
    for v in set(values):
        if values.count(v) == 2:
            return (True, v)
    return (False, 0)

def card_ranking(hand):
    return sorted([card_values.index(card[0]) for card in hand], reverse=True)

def highest_rank(hand):
    if is_royal_flush(hand):
        return (21,0)
    if is_straight_flush(hand):
        return (20, 0)
    is_foak = is_four_of_a_kind(hand)
    if is_foak[0]:
        return (19, is_foak[1])
    is_fh = is_full_house(hand)
    if is_fh[0]:
        return (18,is_fh[1],is_fh[2])
    if is_flush(hand):
        return (17, 0)
    if is_straight(hand):
        return (16, 0)
    is_toak = is_three_of_a_kind(hand)
    if is_toak[0]:
        return (15, is_toak[1])
    is_tp = is_two_pairs(hand)
    if is_tp[0]:
        return (14, is_tp[1][0], is_tp[1][1])
    is_ip = is_pair(hand)
    if is_ip[0]:
        return (13, is_ip[1])
    else:
        return (max(card_ranking(hand)), 0)
    
def winner_by_highest(hand1, hand2):
    rank1 = card_ranking(hand1)
    rank2 = card_ranking(hand2)
    while len(rank1) != 0 and rank1[0] == rank2[0]:
        rank1 = rank1[1:len(rank1)]
        rank2 = rank2[1:len(rank2)]
    if rank1[0] > rank2[0]:
        return 1
    else:
        return 2
    
def determine_winner(hand1, hand2):
    rank1 = highest_rank(hand1)
    rank2 = highest_rank(hand2)
    if rank1[0] > rank2[0]:
        return 1
    elif rank2[0] > rank1[0]:
        return 2
    elif rank1[0] in [19,18,15,14,13]:
        if card_values.index(rank1[1]) > card_values.index(rank2[1]):
            return 1
        elif card_values.index(rank2[1]) > card_values.index(rank1[1]):
            return 2
        elif rank1[0] in [18,14]:
            if card_values.index(rank1[2]) > card_values.index(rank2[2]):
                return 1
            elif card_values.index(rank2[2]) > card_values.index(rank1[2]):
                return 2
    return winner_by_highest(hand1, hand2)
    
wins = 0
for h in hands:
    winner = determine_winner(hands[h][1], hands[h][2])
    if winner == 1:
        wins += 1
        
wins

376

## Problem 55

### Lychrel numbers

In [191]:
def reverse_add(n):
    return n + int(str(n)[::-1])

def is_palindrome(n):
    return str(n) == str(n)[::-1]

def is_lychrel(n):
    for i in range(0,50):
        n = reverse_add(n)
        if is_palindrome(n):
            return False
    return True


a = 0
for n in range(1,10001):
    if is_lychrel(n):
        a += 1
        
a

249

## Problem 56

### Powerful digit sum

In [195]:
def digit_sum(n):
    return sum([int(i) for i in list(str(n))])

max_ds = 0
for a in range(0,100):
    for b in range(0,100):
        ds = digit_sum(a**b)
        if ds > max_ds:
            max_ds = ds
            
max_ds

972

## Problem 57

### Square root convergents

In [24]:
def next_base(base):
    return (base[1], base[0]+2*base[1])

def calculate_root(base):
    return (base[0]+base[1],base[1])

def evaluate(root):
    return len(str(root[0])) > len(str(root[1]))

base = (1,2)
count = 0
for i in range(1000):
    root = calculate_root(base)
    if evaluate(root):
        count += 1
    base = next_base(base)
    
count

153

## Problem 58

### Spiral primes

In [85]:
limit_pow = 9
limit = 10**limit_pow

def prime_sieve(n):
    sieve = [True] * (int(n/2))
    for i in range(3, int(n**0.5)+1, 2):
        if sieve[int(i/2)]:
            sieve[int(i*i/2)::i] = [False] * int((n - i*i - 1) / (2 * i) + 1)
    return [2] + [2 * i + 1 for i in range(1, int(n/2)) if sieve[i]]

primes = frozenset(prime_sieve(limit))

def diagonal_size(side_length):
    return 2*side_length-1

def count_corner_primes(side_length):
    p = 0
    for n in range(1, 4):
        d = side_length**2 - n*(side_length-1)
        if d in primes:
            p += 1
    return p

side_length = 3
diagonal_primes = count_corner_primes(side_length)
while not diagonal_primes/diagonal_size(side_length) < 0.1:
    side_length += 2
    diagonal_primes += count_corner_primes(side_length)

side_length  

26241

## Problem 59

### XOR decryption

In [142]:
with open("p059_cipher.txt") as f:
    cipher = f.read()
    
text = [int(c) for c in cipher.split(',')]

def decrypt(text, key):
    decrypted = []
    for i in range(0, len(text)):
        decrypted.append(text[i] ^ key[i%3])
    return decrypted

keys = list(itertools.product(range(ord('a'), ord('z')+1), repeat=3))
for key in keys:
    decrypted = decrypt(text, key)
    restored = ''.join([chr(c) for c in decrypted])
    if ' the ' in restored:
        break
    
sum(decrypted)

107359

## Problem 60

### Prime pair sets

In [244]:
import itertools

limit_pow = 9
limit = 10**limit_pow
target_length = 5

def prime_sieve(n):
    sieve = [True] * (int(n/2))
    for i in range(3, int(n**0.5)+1, 2):
        if sieve[int(i/2)]:
            sieve[int(i*i/2)::i] = [False] * int((n - i*i - 1) / (2 * i) + 1)
    return [2] + [2 * i + 1 for i in range(1, int(n/2)) if sieve[i]]

primes = frozenset(prime_sieve(limit))
base_primes = [x for x in primes if x < 10**(limit_pow/2 + 1)]
base_primes.remove(2)
base_primes.remove(5)

found = []
index = 0
while len(found) < target_length:
    if len(base_primes) == index:
        index = base_primes.index(found[-1]) + 1
        found = found[:-1]
    match = True
    n = base_primes[index]
    for f in found:
        if (int(str(f)+str(n)) not in primes) or (int(str(n)+str(f)) not in primes):
            match = False
            break
    if match:
        found.append(n)
    index += 1

sum(found)

26033

## Problem 61

### Cyclical figurate numbers

In [515]:
import itertools

triangles = []
squares = []
pentagonals = []
hexagonals = []
heptagonals = []
octagonals = []

n = 1
while True:
    triangle   = int(n*(n+1)/2)
    square     = int(n*n)
    pentagonal = int(n*(3*n-1)/2)
    hexagonal  = int(n*(2*n-1))
    heptagonal = int(n*(5*n-3)/2)
    octagonal  = int(n*(3*n-2))
    
    if triangle >= 10000:
        break

    if triangle >= 1000 and triangle < 10000:
        triangles.append((str(triangle)[0:2], str(triangle)[2:]))
    if square >= 1000 and square < 10000:
        squares.append((str(square)[0:2], str(square)[2:]))
    if pentagonal >= 1000 and pentagonal < 10000:
        pentagonals.append((str(pentagonal)[0:2], str(pentagonal)[2:]))
    if hexagonal >= 1000 and hexagonal < 10000:
        hexagonals.append((str(hexagonal)[0:2], str(hexagonal)[2:]))
    if heptagonal >= 1000 and heptagonal < 10000:
        heptagonals.append((str(heptagonal)[0:2], str(heptagonal)[2:]))
    if octagonal >= 1000 and octagonal < 10000:
        octagonals.append((str(octagonal)[0:2], str(octagonal)[2:]))
        
    n += 1
    
perms = list(itertools.permutations([triangles, squares, pentagonals, hexagonals, heptagonals, octagonals], 6))
perms = perms[:int(len(perms)/6)]
    
can_work = []
for p in perms:
    options = dict()
    for i in range(0,6):
        options[i] = dict()
        for n in p[i]:
            if n[0] not in options[i]:
                options[i][n[0]] = []
            options[i][n[0]].extend([m for m in p[(i+1)%6] if n[1] == m[0]])
            if len(options[i][n[0]]) == 0:
                del options[i][n[0]]
        
    for k in range(0,6):
        if len(options) == 6:
            for i in range(0,6):
                for n in options[i]:
                    options[i][n] = [m for m in options[i][n] if (m[0] in options[(i+1)%6]) and (m[1] in options[(i+2)%6])]
                options[i] = {n:options[i][n] for n in options[i] if len(options[i][n]) != 0}
            options = {k:options[k] for k in options if len(options[k]) != 0}
        
    if len(options) == 6: 
        can_work.append(options)

can_work = can_work[0]
for n in range(0,6):
    reverse = {}
    for i in range(0,6):
        reverse[i] = {can_work[i][l][0][0] for l in can_work[i]}
    
    for i in range(0,6):
        can_work[i] = {j:can_work[i][j] for j in can_work[i] if j in reverse[(i-1)%6]}

numbers = []
for k in can_work:
    for l in can_work[k]:
        numbers.append(int(l + can_work[k][l][0][0]))
        
sum(numbers)

28684

## Problem 62

### Cubic permutations

In [48]:
import itertools

limit = 10000
cubes = [i**3 for i in range(1,limit)]
perms = dict()
permutations = 5

for c in cubes:
    tmp = list(str(c))
    tmp.sort()
    key = ''.join(tmp)
    if key not in perms:
        perms[key] = []
    perms[key].append(c)
    
for k in perms:
    if len(perms[k]) >= permutations:
        break
        
perms[k][0]

127035954683

## Problem 63

### Powerful digit counts

In [47]:
import math

count = 0
n = 1
min_pow = 1
while min_pow < 10:
    min_pow = math.ceil((10**(n-1))**(1/n))
    count += 10 - min_pow
    n += 1  

count

49

## Problem 64

### Odd period square roots

In [129]:
import math

def square_base(n):
    return int(math.sqrt(n))

def simplify(n,d):
    i = 2
    while i <= n and i <= d:
        while n % i == 0 and d % i == 0:
            n = n/i
            d = d/i
        i = i + 1
    return (int(n), int(d))

def next_fraction(n, base, nom, den_minus):
    next_nom = n - den_minus * den_minus
    nom, next_nom = simplify(nom, next_nom)
    next_part = int((base + den_minus) / next_nom)
    next_den_minus = -(den_minus - next_part * next_nom)
    return (next_nom, next_den_minus)

def period_length(n):
    previous = set()
    base = square_base(n)
    nom, den_minus = (1, base)

    while (nom, den_minus) not in previous:
        previous.add((nom, den_minus))
        nom, den_minus = next_fraction(n, base, nom, den_minus)
        
    return len(previous)

count = 0
for n in range(2, 10000):
    if math.sqrt(n) != square_base(n):
        length = period_length(n)
        if length % 2 == 1:
            count += 1
            
count

1322

## Problem 65

### Convergents of e

In [97]:
import math

parts = []
for i in range(1, 34):
    parts.extend([1,1,i*2])
parts[0] = 2
parts.append(1)

def calculate(parts):
    parts.reverse()
    num, den = (parts[0], 1)
    for p in parts[1:]:
        num, den = (den, num)
        num += den * p
    return num, den

def digit_sum(n):
    return sum([int(i) for i in list(str(n))])

fraction = calculate(parts)
digit_sum(fraction[0])

272

## Problem 66

### Diophantine equation

In [190]:
import math

def square_base(n):
    return int(math.sqrt(n))

def simplify(n,d):
    i = 2
    while i <= n and i <= d:
        while n % i == 0 and d % i == 0:
            n = n/i
            d = d/i
        i = i + 1
    return (int(n), int(d))

def next_fraction(n, base, nom, den_minus):
    next_nom = n - den_minus * den_minus
    nom, next_nom = simplify(nom, next_nom)
    next_part = int((base + den_minus) / next_nom)
    next_den_minus = -(den_minus - next_part * next_nom)
    return (next_part, next_nom, next_den_minus)

def calculate(parts):
    parts.reverse()
    num, den = (parts[0], 1)
    for p in parts[1:]:
        num, den = (den, num)
        num += den * p
    parts.reverse()
    return num, den

def find_solution(D):
    base = square_base(D)
    nom, den_minus = (1, base)
    parts = [base]
    x, y = (1, base)
    while x*x - D*y*y != 1:
        part, nom, den_minus = next_fraction(D, base, nom, den_minus)
        parts.append(part)
        x, y = calculate(parts)
    return x

solutions = {}
for D in range(2, 1000):
    sqrt = math.sqrt(D)
    if sqrt != int(sqrt):
        x = find_solution(D)
        solutions[x] = D

solutions[max(solutions)]

661

## Problem 67

### Maximum path sum II

In [257]:
with open("p067_triangle.txt") as f:
    data = f.read()
    
triangle = [[int(n) for n in row.split(' ')] for row in data.split('\n')[:-1]]

def calculate_current(level, i):
    current = triangle[level][i]
    left = triangle[level+1][i]
    right = triangle[level+1][i+1]
    return current + max(left, right)

for level in range(len(triangle)-2, -1, -1):
    for i in range(0,len(triangle[level])):
        triangle[level][i] = calculate_current(level, i)
        
triangle[0][0]

7273